In [1]:
!pip install tensorflow_text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 49.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fou

In [2]:
!pip uninstall tensorflow_hub


Found existing installation: tensorflow-hub 0.14.0
Uninstalling tensorflow-hub-0.14.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tensorflow_hub-0.14.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow_hub/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-hub-0.14.0


In [3]:
!pip install tensorflow_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.3/90.3 kB 1.4 MB/s eta 0:00:00


In [4]:
import tensorflow_text as text
import tensorflow_hub as hub

In [5]:
import pandas as pd

In [7]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.Category.value_counts() #as the category values are not equal we are gonna do downsampling just to keep simple

ham     4825
spam     747
Name: Category, dtype: int64

In [9]:
df_spam = df[df['Category']=='spam']

In [10]:
df_spam.shape

(747, 2)

In [11]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [12]:
df_ham_downsampled = df_ham.sample(747)
df_ham_downsampled.shape

(747, 2)

In [13]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(1494, 2)

In [14]:
df_balanced.head()

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [15]:
df_balanced['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0) #apply is used for data transform where u can enter any function

In [16]:
df_balanced.sample(20)

,Category,Message,spam
2002,ham,Idea will soon get converted to live:),0
42,spam,07732584351 - Rodger Burns - MSG = We tried to...,1
1830,spam,Hottest pics straight to your phone!! See me g...,1
4261,ham,Are you plans with your family set in stone ?,0
955,spam,Filthy stories and GIRLS waiting for your,1
1430,spam,For sale - arsenal dartboard. Good condition b...,1
4602,spam,FreeMsg: Txt: CALL to No: 86888 & claim your r...,1
4766,ham,if you text on your way to cup stop that shoul...,0
1175,ham,Yay! You better not have told that to 5 other ...,0
2095,spam,PRIVATE! Your 2004 Account Statement for 07742...,1


In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_balanced.Message, df_balanced.spam, stratify=df_balanced.spam)

#stratify is used for splittng equal amount of spam and ham in the train test split

In [18]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [19]:
def get_embeddings(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

In [20]:
get_embeddings([
    "500$ discount. hurry up",
    "Bhavin, are you up for volleyall game tomorrow?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351724, -0.5132727 , -0.88845736, ..., -0.7474883 ,
        -0.75314754,  0.91964495],
       [-0.84260833, -0.4590629 , -0.9361199 , ..., -0.85912603,
        -0.6814544 ,  0.8538231 ]], dtype=float32)>

In [21]:
e =  get_embeddings([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [22]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.7606916 , -0.14219397,  0.49604574, ...,  0.42165306,
        -0.5322139 ,  0.80312157],
       [-0.86023194, -0.21242912,  0.4915695 , ...,  0.39798063,
        -0.60506254,  0.8447163 ],
       [-0.7128858 , -0.15463883,  0.38401723, ...,  0.35278767,
        -0.50991315,  0.73474056],
       [-0.82533485, -0.35550627, -0.5906983 , ..., -0.01613823,
        -0.614176  ,  0.872303  ],
       [-0.75041324, -0.2681263 , -0.26689714, ...,  0.02839372,
        -0.59380966,  0.7974984 ],
       [-0.785443  , -0.29949623,  0.4102765 , ...,  0.52225506,
        -0.4957351 ,  0.81507415]], dtype=float32)>

In [23]:
import tensorflow as tf


In [26]:
#This is functional way of creating neural network
#bert layer
input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text" )#input layer
preprocessed_text = bert_preprocess(input_layer)
outputs = bert_encoder(preprocessed_text)

In [32]:
#Neural Network
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])#ending should be the previous line
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

#Completing final model
model = tf.keras.Model(inputs=[input_layer], outputs=[l])

In [33]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [34]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = METRICS)

In [35]:
model.fit(x_train,y_train, epochs = 10)

Epoch 1/10
35/35 [==============================] - 23s 329ms/step - loss: 0.6509 - accuracy: 0.6205 - precision: 0.6212 - recall: 0.6179
Epoch 2/10
35/35 [==============================] - 11s 324ms/step - loss: 0.5275 - accuracy: 0.7946 - precision: 0.7835 - recall: 0.8143
Epoch 3/10
35/35 [==============================] - 12s 337ms/step - loss: 0.4527 - accuracy: 0.8527 - precision: 0.8459 - recall: 0.8625
Epoch 4/10
35/35 [==============================] - 12s 343ms/step - loss: 0.4044 - accuracy: 0.8616 - precision: 0.8438 - recall: 0.8875
Epoch 5/10
35/35 [==============================] - 13s 378ms/step - loss: 0.3655 - accuracy: 0.8813 - precision: 0.8700 - recall: 0.8964
Epoch 6/10
35/35 [==============================] - 12s 345ms/step - loss: 0.3364 - accuracy: 0.8884 - precision: 0.8769 - recall: 0.9036
Epoch 7/10
35/35 [==============================] - 12s 341ms/step - loss: 0.3275 - accuracy: 0.8911 - precision: 0.8925 - recall: 0.8893
Epoch 8/10
35/35 [================

In [36]:
model.evaluate(x_test,y_test)

12/12 [==============================] - 7s 322ms/step - loss: 0.2463 - accuracy: 0.9332 - precision: 0.9263 - recall: 0.9412


[0.24628619849681854,
 0.9331550598144531,
 0.9263157844543457,
 0.9411764740943909]

In [37]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Dgital Camera! call 0901328555 from landline. Delivery within 128 days. 2yr warranty',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005',
    'Hey Sam, Are you coming  for a cricket game tomorrow',
    "why don't you wait till at least wednesday to see if you get your"
]
model.predict(reviews)

1/1 [==============================] - 1s 1s/step


array([[0.7534224 ],
       [0.74677986],
       [0.8743129 ],
       [0.17942865],
       [0.08417215]], dtype=float32)